## Load Packages

In [1]:
using StatsBase

include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Loading Daily S&P 500 Data

In [3]:
xx = readdlm("Data/SP500RfPs.csv",',',header=true)
x  = xx[1]
SP = convert(Array{Float64},x[:,2])      #convert to numerical array, S&P 500 level
R  = (SP[2:end]./SP[1:end-1] - 1) * 100  #returns, % 
T  = length(R)  

dN = Date.(string.(x[2:end,1]),"d/m/y")    #convert to Date, 2:end as for R
println("Number of days in the sample: ",T)

Number of days in the sample: 9352


# Autocorrelations

$\rho_s = \text{Corr}(R_t,R_{t-s})$

In large samples, $\sqrt{T}\hat{\rho}_{s}\sim N(0,1)$, if the true value is $\rho_s=0$ for all $s$

In [4]:
plags = 1:5
ρ = autocor(R,plags)
println("\nlag, autocorrr, t-stat(autocorr) of excess returns")
printmat([plags ρ sqrt(T)*ρ])


lag, autocorrr, t-stat(autocorr) of excess returns
     1.000    -0.026    -2.480
     2.000    -0.044    -4.208
     3.000    -0.007    -0.684
     4.000    -0.022    -2.124
     5.000    -0.018    -1.744



# Autoregressions

An AR(1) is 

$R_{t}=c+a_{1}R_{t-1}+\varepsilon_{t}$

and can be estimated by OLS

An asymmteric AR(1) 

$R_{t} =\alpha+\beta Q_{t-1}R_{t-1}+\gamma(1-Q_{t-1})R_{t-1}+\varepsilon
_{t}$,

where $Q_{t-1}=1 \ \text{ if } \ R_{t-1}<0$ and zero otherwise

In [5]:
function OlsFn(y,x)                      #OLS estimation, alternative: use GLM.jl 
    b     = x\y
    yhat  = x*b
    res   = y - yhat
    Covres = cov(res)
    Covb   = kron(Covres,inv(x'x))       #works also if y is Txn
    R2a    = 1 - var(res,1)./var(y,1)
    return b,res,yhat,Covb,R2            #the outputs
end

OlsFn (generic function with 1 method)

In [6]:
y  = R[2:end]
Tb = size(y,1)
x  = [ones(Tb) R[1:end-1]]
(b,res,yhat,Covb,) = OlsFn(y,x) 
Stdb  = sqrt.(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1): intercept and slope with t-stats")
printmat([b tstat])

Results from AR(1): intercept and slope with t-stats
     0.039     3.405
    -0.026    -2.481



In [7]:
DummyNegative = R[1:end-1] .< 0

x = [ones(Tb) DummyNegative.*R[1:end-1] (1-DummyNegative).*R[1:end-1]]

(b,res,yhat,Covb,) = OlsFn(y,x) 
Stdb  = sqrt.(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1) with dummies: [intercept, slope neg, slope pos] with t-stats")
printmat([b tstat])

Results from AR(1) with dummies: [intercept, slope neg, slope pos] with t-stats
     0.001     0.058
    -0.075    -4.395
     0.027     1.518



## Recursive Estimation and Out-of-Sample R2

Do recursive estimation (longer and longer sample) an predict one period ahead (outside of the sample). Define an "out-of-sample" $R^2$ as 

$R_{OS}^2 = 1- \frac{\text{MSE(forecasting model)}}{\text{MSE(benchmark forecast)}}$

In [8]:
y = R[2:end]
Tb = size(y,1)
x = [ones(Tb) R[1:end-1]]

bT     = fill(NaN,size(x))
rHat   = fill(NaN,Tb)
rTilde = fill(NaN,Tb)
for t = 100:Tb
    (b,res,yhat,Covb,) = OlsFn(y[1:t-1],x[1:t-1,:]) 
    bT[t,:] = b'
    rHat[t] = (x[t:t,:]*b)[1]           #out-of-sample forecast of period t
    rTilde[t] = mean(y[1:t-1])          #benchmark forecast: historical average
end    

MSE_Model = mean((y[100:end]-rHat[100:end]).^2)    #MSE for out-of-sample period
MSE_Bench = mean((y[100:end]-rTilde[100:end]).^2)

printlnPs("MSE of AR(1) model and the benchmark: ",[MSE_Model MSE_Bench])
printlnPs("out-of-sample R2: ",1-MSE_Model/MSE_Bench)

MSE of AR(1) model and the benchmark:      1.247     1.244
out-of-sample R2:     -0.002


In [9]:

plot(dN[2:end],bT[:,1],color=:blue,legend=false)
title!("Estimated intercept")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980-01-01 
 
 
 1989-01-01 
 
 
 1998-01-01 
 
 
 2007-01-01 
 
 
 2016-01-01 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 Estimated intercept 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.1267,282.626 45.29,268.686 45.3308,279.478 45.3716,329.208 45.4125,327.983 45.5349,323.931 45.5758,317.399 45.6166,254.475 45.6574,230.703 45.6983,213.023 
 45.8207,230.121 45.8616,209.856 45.9024,171.513 45.9432,198.409 45.984,203.477 46.1065,209.817 46.1474,235.16 46.1882,233.528 46.229,232.777 46.2698,214.248 
 46.3923,194.779 46.4331,220.186 46.474,239.202 46.5148,211.738 46.5556,194.528 46.6781,193.605 46.7189,232.794 46.8006,226.122 46.8414,214.124 46.9639,170.725 
 47.0047,147.816 47.0455,161.719 47.0864,185.06 47.1272,223.861 47.2497,236.561 47.2905,219.234 47.3313,255.103 47.3722,257.944 47.413,261.434 47.5355,254.138 
 47.5763,264.351 47.6171,249.991 47.6579,213.397 47.6988,217.886 47.8213,219.209 47.8621,218.662 47.9029,196.953 47.9437,189.109 47.9846,194.099 48.107,197.281 
 48.1479,189.471 48.1887,147.829 48.2295,144.702 48.2704,162.786 48.3928,130.396 48.4337,106.545 48.4745,108.742 48.5153,88.2326 48.5561,96.679 48.6786,91.3261 
 48.7194,78.1791 48.7603,79.5072 48.8011,79.3563 48.8419,92.2753 48.9644,93.9023 49.0052,79.455 49.0461,86.0528 49.0869,87.572 49.1277,89.4418 49.291,82.5724 
 49.3319,136.312 49.3727,169.936 49.4135,151.841 49.536,132.951 49.5768,124.256 49.6176,144.459 49.6585,133.841 49.6993,135.492 49.8218,111.93 49.8626,114.054 
 49.9034,138.458 49.9443,128.304 49.9851,73.0169 50.1076,79.2015 50.1484,103.627 50.1892,100.401 50.23,94.9617 50.2709,91.1272 50.3934,115.371 50.4342,136.562 
 50.475,105.848 50.5158,109.561 50.5567,96.377 50.6791,74.2848 50.72,105.932 50.7608,202.139 50.8016,233.361 50.8424,234.813 50.9649,249.083 51.0058,277.377 
 51.0466,276.972 51.0874,271.599 51.1282,267.653 51.2507,318.877 51.2915,334.385 51.3324,341.609 51.3732,335.537 51.414,347.684 51.5365,331.026 51.5773,330.576 
 51.6182,283.018 51.659,309.693 51.6998,287.274 51.8223,291.942 51.8631,308.855 51.9039,322.031 51.9448,352.931 51.9856,336.42 52.1081,309.711 52.1489,270.209 
 52.1897,289.816 52.2306,277.186 52.2714,262.891 52.3939,273.516 52.4347,262.537 52.4755,276.482 52.5572,270.311 52.6797,254.361 52.7205,213.257 52.7613,227.969 
 52.8021,218.491 52.843,219.709 52.9654,234.423 53.0063,240.247 53.0471,218.753 53.0879,213.537 53.1288,200.748 53.2512,213.097 53.2921,209.027 53.3329,213.933 
 53.3737,213.488 53.4145,211.291 53.537,187.506 53.5779,184.611 53.6187,206.296 53.6595,205.814 53.7003,205.078 53.8228,219.212 53.9045,216.299 53.9453,214.883 
 53.9861,212.475 54.1086,216.042 54.1903,214.431 54.2311,257.257 54.2719,262.6 54.3944,234.867 54.4352,233.912 54.476,195.162 54.5169,199.49 54.5577,184.949 
 54.6802,187.41 54.721,179.838 54.7618,168.589 54.8027,172.877 54.8435,179.86 54.966,172.799 55.0068,156.83 55.0476,169.468 55.0884,132.468 55.1293,133.508 
 55.2518,136.588 55.2926,115.776 55.3334,130.551 55.3742,108.802 55.4151,127.021 55.5375,107.73 55.5784,121.444 55.6192,115.797 55.66,100.222 55.7009,94.7739 
 55.8233,71.3134 55.8642,85.244 55.905,71.1427 55.9458,65.5713 55.9866,92.8557 56.15,115.933 56.1908,129.295 56.2316,96.0006 56.2724,114.198 56.3949,117.628 
 56.4357,146.094 56.4766,131.914 56.5174,157.456 56.5582,156.466 56.6807,136.217 56.7215,154.526 56.7624,148.994 56.8032,175.514 56.844,219.131 56.9665,248.771 
 57.0073,252.864 57.0481,231.243 57.089,247.408 57.1298,267.742 57.2523,269.116 57.2931,321.633 57.3339,281.003 57.3748,280.413 57.4156,300.375 57.5381,313.09 
 57.5789,363.895 57.6197,360.561 57.6605,369.211 57.7014,376.378 57.8239,333.502 57.8647,315.216 57.9055,316.342 57.9463,308.641 58.1096,318.12 58.1505,349.659 
 58.1913,329.058 58.2321,300.688 58.273,289.583 58.3954,295.887 58.4363,310.729 58.4771,312.739 58.5179,329.885 58.

In [10]:
plot(dN[2:end],bT[:,2],color=:red,legend=false)
title!("Estimated AR(1) slope")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980-01-01 
 
 
 1989-01-01 
 
 
 1998-01-01 
 
 
 2007-01-01 
 
 
 2016-01-01 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 Estimated AR(1) slope 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.1267,228.44 45.29,228.384 45.3308,230.454 45.3716,222.988 45.4125,223.809 45.5349,223.975 45.5758,223.654 45.6166,217.671 45.6574,192.336 45.6983,183.502 
 45.8207,189.311 45.8616,194.253 45.9024,183.279 45.9432,203.009 45.984,200.986 46.1065,200.251 46.1474,197.308 46.1882,197.963 46.229,197.977 46.2698,197.9 
 46.3923,191.467 46.4331,201.761 46.474,193.724 46.5148,203.169 46.5556,195.186 46.6781,194.825 46.7189,196.56 46.8006,200.971 46.8414,200.46 46.9639,190.241 
 47.0047,166.716 47.0455,176.036 47.0864,170.532 47.1272,154.014 47.2497,145.564 47.2905,150.751 47.3313,161.394 47.3722,159.543 47.413,159.074 47.5355,159.73 
 47.5763,161.104 47.6171,164.018 47.6579,153.906 47.6988,158.125 47.8213,158.097 47.8621,158.117 47.9029,158.076 47.9437,154.042 47.9846,155.279 48.107,154.977 
 48.1479,155.659 48.1887,148.687 48.2295,144.795 48.2704,148.621 48.3928,161.24 48.4337,143.4 48.4745,145.277 48.5153,144.888 48.5561,149.575 48.6786,150.488 
 48.7194,149.156 48.7603,149.647 48.8011,149.648 48.8419,149.626 48.9644,149.123 49.0052,150.305 49.0461,152.87 49.0869,152.643 49.1277,152.529 49.291,152.95 
 49.3319,162.626 49.3727,125.306 49.4135,138.222 49.536,132.304 49.5768,126.743 49.6176,133.966 49.6585,139.161 49.6993,139.476 49.8218,140.157 49.8626,141.731 
 49.9034,141.848 49.9443,147.895 49.9851,138.72 50.1076,154.426 50.1484,152.355 50.1892,154.199 50.23,154.224 50.2709,153.648 50.3934,156.724 50.4342,144.116 
 50.475,160.323 50.5158,163.348 50.5567,164.035 50.6791,155.32 50.72,183.191 50.7608,104.325 50.8016,57.0226 50.8424,55.9032 50.9649,53.2496 51.0058,42.767 
 51.0466,43.0468 51.0874,43.7592 51.1282,43.3283 51.2507,49.6016 51.2915,31.6297 51.3324,27.5591 51.3732,29.3062 51.414,30.4276 51.5365,35.3641 51.5773,35.1965 
 51.6182,31.4502 51.659,69.1282 51.6998,80.8387 51.8223,83.0556 51.8631,82.1019 51.9039,76.3963 51.9448,64.2005 51.9856,77.5246 52.1081,70.3952 52.1489,38.9552 
 52.1897,65.7875 52.2306,70.128 52.2714,66.4723 52.3939,71.1509 52.4347,73.5316 52.4755,76.9815 52.5572,79.0125 52.6797,77.417 52.7205,58.0882 52.7613,79.7559 
 52.8021,82.1658 52.843,82.4103 52.9654,82.3396 53.0063,80.1534 53.0471,84.6434 53.0879,81.6631 53.1288,78.5406 53.2512,83.8028 53.2921,84.9738 53.3329,85.2546 
 53.3737,85.3121 53.4145,85.3374 53.537,83.9507 53.5779,81.6917 53.6187,86.214 53.6595,86.4825 53.7003,86.5458 53.8228,86.6101 53.9045,87.745 53.9453,87.7276 
 53.9861,87.6276 54.1086,87.9027 54.1903,88.0604 54.2311,89.3434 54.2719,83.5112 54.3944,92.4949 54.4352,91.7187 54.476,86.1047 54.5169,92.3744 54.5577,91.9986 
 54.6802,93.1473 54.721,93.2581 54.7618,90.6705 54.8027,92.4396 54.8435,91.8721 54.966,93.3953 55.0068,90.5793 55.0476,98.0504 55.0884,110.366 55.1293,111.682 
 55.2518,111.998 55.2926,113.74 55.3334,124.299 55.3742,133.157 55.4151,144.68 55.5375,154.592 55.5784,161.664 55.6192,163.871 55.66,162.058 55.7009,158.899 
 55.8233,152.703 55.8642,166.788 55.905,172.622 55.9458,170.474 55.9866,177.084 56.15,160.01 56.1908,151.968 56.2316,165.537 56.2724,186.143 56.3949,184.262 
 56.4357,179.973 56.4766,190.597 56.5174,198.532 56.5582,199.238 56.6807,200.15 56.7215,213.125 56.7624,216.123 56.8032,219.076 56.844,186.957 56.9665,155.973 
 57.0073,152.733 57.0481,157.231 57.089,167.998 57.1298,159.014 57.2523,158.198 57.2931,152.401 57.3339,208.273 57.3748,207.709 57.4156,209.455 57.5381,202.435 
 57.5789,182.676 57.6197,187.135 57.6605,186.717 57.7014,184.941 57.8239,194.6 57.8647,172.05 57.9055,172.8 57.9463,172.559 58.1096,174.648 58.1505,166.913 
 58.1913,184.403 58.2321,171.203 58.273,161.207 58.3954,163.903 58.4363,161.89 58.4771,161.065 58.5179,159.